#### TOOLS, MODULES INSTALLATION

In [21]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install pyswarm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
import numpy as np
import pandas as pd
import yfinance as yf
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import binarize, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from pyswarm import pso

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### DATA PREPROCESSING - NORMALIZATION


In [24]:
original_data = pd.read_csv('/content/drive/MyDrive/classes_NUS/Project_Module_1/preprocessed_data.csv')
original_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date_x,Open,High,Low,Close,Adj Close,Volume,Ticker,Beta,P/E Ratio,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
0,2020-01-02,-0.527490,-0.528801,-0.526355,-0.527421,-0.529219,-0.177086,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,3,-0.000131,NaN,NaN,0.710776,-0.2276
1,2020-01-03,-0.527319,-0.528716,-0.526527,-0.527676,-0.529440,-0.103929,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,4,0.000677,NaN,NaN,0.710776,-0.2276
2,2020-01-06,-0.527745,-0.529055,-0.526527,-0.527847,-0.529587,-0.133960,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,0,0.000192,NaN,NaN,0.710776,-0.2276
3,2020-01-07,-0.527831,-0.529139,-0.526871,-0.527761,-0.529513,0.027768,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,1,-0.000131,NaN,NaN,0.710776,-0.2276
4,2020-01-08,-0.528257,-0.528885,-0.526871,-0.527506,-0.529293,0.073270,Z74.SI,-0.33756,1.03557,...,-1.746289,-1.285688,0.711104,1,2,-0.001421,NaN,NaN,0.710776,-0.2276


In [25]:
# get the columns with the non numeric values
nncol = original_data.select_dtypes(exclude=['number'])
print(nncol.columns)

#drop the non numeric columns
col_drp = nncol.columns
temp_data = original_data.drop(columns = col_drp)

# get the columns with NaN values or Null values
if temp_data.isna().any().any():
  temp_data.fillna(0, inplace=True)

temp_data.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Date_x', 'Ticker'], dtype='object')


,Open,High,Low,Close,Adj Close,Volume,Beta,P/E Ratio,Market Cap,52 Week High,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
count,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,6.634300e+04,66343.000000,...,6.634300e+04,6.634300e+04,6.634300e+04,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000,66343.000000
mean,-0.009757,-0.009441,-0.010087,-0.009760,-0.010201,-0.060291,-0.007953,-0.037481,2.786560e+11,94.899248,...,9.262128e-16,1.765031e-16,1.418022e-16,6.274392,2.015058,-0.001192,-0.009659,-0.009499,-0.375816,-0.169080
std,0.966994,0.968124,0.965806,0.966983,0.965142,0.570753,0.973993,0.847904,7.811223e+11,167.325227,...,1.000008e+00,1.000008e+00,1.000008e+00,3.380699,1.399944,0.367440,0.965551,0.963438,0.865194,0.431816
min,-0.556234,-0.557377,-0.555149,-0.556247,-0.554060,-0.328958,-1.433540,-0.880751,1.000000e-05,0.060000,...,-1.746289e+00,-1.285688e+00,-1.279092e+00,1.000000,0.000000,-67.852792,-0.556209,-0.556199,-2.126542,-1.257573
25%,-0.543084,-0.544188,-0.542019,-0.543099,-0.542182,-0.305161,-0.459101,-0.558389,1.556755e+09,1.580000,...,-6.457594e-02,-6.704259e-01,-1.008615e+00,3.000000,1.000000,-0.000294,-0.543061,-0.542935,-0.817326,-0.314209
50%,-0.525359,-0.526511,-0.524276,-0.525448,-0.525623,-0.262832,-0.154182,-0.182961,7.518913e+09,3.760000,...,3.617085e-01,-1.217801e-01,7.111045e-01,6.000000,2.000000,0.000026,-0.525341,-0.525230,-0.202068,-0.223999
75%,0.113090,0.116107,0.108516,0.112495,0.120898,-0.120878,0.476978,0.026823,2.066344e+10,112.790000,...,3.617085e-01,1.238477e+00,9.498384e-01,9.000000,3.000000,0.000499,0.117040,0.117593,0.026470,-0.096765
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.439320e+12,577.470000,...,1.188818e+00,1.238477e+00,9.498384e-01,12.000000,4.000000,15.927395,3.000000,3.000000,1.386294,1.386294


In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_data = scaler.fit_transform(temp_data)
# the output from scaler normalisation is is ndarray, convert to pd
normalized_data = pd.DataFrame(normalized_data, columns=temp_data.columns)
normalized_data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Open,High,Low,Close,Adj Close,Volume,Beta,P/E Ratio,Market Cap,52 Week High,...,PC1,PC2,PC3,Month,DayOfWeek,Daily_Return,MA20,MA50,Log_PE_Ratio,Log_PEG_Ratio
0,-0.535409,-0.536465,-0.534550,-0.535340,-0.537768,-0.204634,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,0.703563,0.002887,0.010004,0.009859,1.255903,-0.13552
1,-0.535232,-0.536377,-0.534728,-0.535604,-0.537996,-0.076458,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,1.417883,0.005086,0.010004,0.009859,1.255903,-0.13552
2,-0.535673,-0.536727,-0.534728,-0.535780,-0.538148,-0.129074,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-1.439395,0.003766,0.010004,0.009859,1.255903,-0.13552
3,-0.535761,-0.536814,-0.535084,-0.535692,-0.538072,0.154286,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-0.725076,0.002887,0.010004,0.009859,1.255903,-0.13552
4,-0.536202,-0.536552,-0.535084,-0.535428,-0.537844,0.234011,-0.338411,1.265543,-0.288668,-0.546779,...,-1.746289,-1.285688,0.711104,-1.56016,-0.010756,-0.000625,0.010004,0.009859,1.255903,-0.13552


#### DATA PREPROCESSING - BINARY DATA FOR APRIORI ALGORITHM


In [27]:
#Get transposed, binary data for Apriori algorithm
def process_data_binary_tranpose(normalized_data, original_data):

    temp = normalized_data
    temp['Ticker'] = original_data['Ticker']
    temp_gp = temp.groupby('Ticker').mean() #group by makes the Ticker as index and not an active column,
    temp_gp_bin = temp_gp.pct_change().apply(lambda x: (x > 0).astype(int))
    temp_gp_bin_tr_data = temp_gp_bin.T
    temp_gp_bin_tr_data.columns = temp_gp_bin.index
    temp_gp_bin_tr_data = temp_gp_bin_tr_data.reset_index() #need to apply reset_index for making use of the tickers rows (transpose done)
    temp_gp_bin_tr_data = temp_gp_bin_tr_data.drop(columns=['index'])
    binary_transposed_data = temp_gp_bin_tr_data

    return binary_transposed_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### GET THE CLUSTERS FOR PORTFOLIO CONSTRUCTION USING RULE BASED CLUSTERING ALGORITHM - APRIORI, ASSOCIATION RULES

In [28]:
#Apply Apriori algorithm for rule-based clustering,
#min_supprt value here is 0.43, because,
#less than 0.43 creates thousands of rules
#and greater than 0.5 creates no rules
def rule_based_clusters(binary_transposed_data, min_support=0.43, min_confidence=0.43):

    frequent_itemsets = apriori(binary_transposed_data, min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    clusters = {}
    index = 0
    for each_rule in rules.itertuples():
      if len(each_rule.consequents) > 2:
        print(each_rule.consequents)
        clusters[index] = list(each_rule.consequents)
        index = index + 1
      else:
        continue

    for key, value in clusters.items():
        print(f"{key}: {value}")

    return clusters

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### DIFFERENT TYPES OF PORTFOLIO CREATION FROM THE CLUSTERS

In [29]:
def create_portfolios(normalized_data_with_ticker, clusters, portfolio_type_weights = 'EW'):
    portfolios = {}
    # Iterate over each cluster
    for cluster_id, tickers in clusters.items():

        # Extract the stock tickers (values in the 'Ticker' column)
        cluster_data = normalized_data_with_ticker[normalized_data_with_ticker['Ticker'].isin(tickers)]
        cluster_data_n = cluster_data.drop(columns=['Ticker'])
        num_ticker = len(tickers)

        # Equal Weighted (EW) Portfolio
        if portfolio_type_weights == 'EW':
          ew_weights = np.ones(num_ticker) / num_ticker
          portfolio_name = f'EW_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, ew_weights))

        elif portfolio_type_weights == 'MCW':
          # Market Cap Weighted (MCW) Portfolio (Simulated Market Cap for simplicity)
          market_caps = np.random.uniform(100, 500, num_ticker)
          mcw_weights = market_caps / np.sum(market_caps)
          portfolio_name = f'MCW_Cluster_{cluster_id}'
          portfolios[portfolio_name] = mcw_weights
          portfolios[portfolio_name] = dict(zip(tickers, mcw_weights))

        elif portfolio_type_weights == 'GMV':
          # Global Minimum Variance (GMV) Portfolio
          returns = cluster_data_n.pct_change().dropna()
          cov_matrix = returns.cov()
          inv_cov_matrix = np.linalg.inv(cov_matrix)
          ones = np.ones(cov_matrix.shape[0])
          gmv_weights = np.dot(inv_cov_matrix, ones) / np.dot(ones.T, np.dot(inv_cov_matrix, ones))
          portfolio_name = f'GMV_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, gmv_weights))

        elif portfolio_type_weights == 'MSR':
          # Maximum Sharpe Ratio (MSR) Portfolio
          cluster_n = cluster_data_n.pct_change().dropna()
          cluster_exp_returns = cluster_n.mean() * 252  # Annualized returns
          cluster_exp_returns = pd.DataFrame(cluster_exp_returns)
          cov_matrix = cluster_n.cov()
          inv_cov_matrix = np.linalg.inv(cov_matrix)
          ones = np.ones((len(cluster_exp_returns), 1))
          numer = np.dot(inv_cov_matrix, cluster_exp_returns)
          denom = np.dot(ones.T, numer)
          msr_weights = numer / denom
          portfolio_name = f'MSR_Cluster_{cluster_id}'
          portfolios[portfolio_name] = dict(zip(tickers, msr_weights.flatten()))
        else:
            print("Invalid portfolio_type_weights set")
            return None
    return portfolios

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### OBJECTIVE FUNCTION

In [30]:
def objective_function(weights, returns, cov_matrix, risk_free_rate=0.01):

    weights = weights.flatten()

    if isinstance(returns, (int, float)):  # Check if returns is a scalar
      portfolio_return = returns * weights[0]
      portfolio_volatility = np.sqrt(cov_matrix * (weights[0]**2))  # Assume cov_matrix is also a scalar
    else:
      # Portfolio returns
      portfolio_return = np.sum(returns * weights)
      # Portfolio volatility (risk)
      portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    if portfolio_volatility == 0:
        portfolio_volatility = np.inf  # Avoid division by zero

    print(f"portfolio_volatility : {portfolio_volatility}")
    print(f"portfolio_return : {portfolio_return}")
    # Sharpe Ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

    # We want to maximize Sharpe Ratio and minimize volatility
    return sharpe_ratio#, #portfolio_volatility

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### PSO OPTIMISATION


In [31]:
def pso_optimization(returns, cov_matrix):
    num_assets = len(returns)
    bounds = [(0, 1) for _ in range(num_assets)]
    lb = [0] * num_assets
    ub = [1] * num_assets
    optimal_weights, _ = pso(objective_function, lb, ub,
                             args=(returns.values, cov_matrix.values), # Pass returns and cov_matrix as NumPy arrays
                             swarmsize=10, maxiter=10)

    # Normalize weights to sum to 1
    optimal_weights /= np.sum(optimal_weights)

    return optimal_weights

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### LSTM FORECAST

In [32]:


# LSTM for stock price forecasting
def lstm_forecast(prices, portfolio_cluster, look_back=60):

    # Scaling data between 0 and 1 for LSTM
    dataset = prices.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # Preparing the data for LSTM
    X, y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i - look_back:i])  # Sequence of past 'look_back' days
        y.append(dataset[i])  # Predict next value based on 'look_back' values

    X, y = np.array(X), np.array(y)

    # Get the number of features (columns) in prices data
    num_features = prices.shape[1]

    # Reshape X to the required shape for LSTM input (samples, time steps, features)
    X = X.reshape(X.shape[0], X.shape[1], num_features)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], num_features)))  # Update the input shape to match number of features
    model.add(Dropout(0.2))
    model.add(LSTM(units=10, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=num_features))  # Predicting the same number of features as input

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=16)

    # Predicting future prices
    predicted_prices = model.predict(X_test)

    # Inverse scaling to get the actual prices
    predicted_prices = scaler.inverse_transform(predicted_prices)

    model.save_weights('model_weights_' + portfolio_cluster + '.weights.h5')

    return predicted_prices

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### MAIN API - ACTIVE PORTFOLIO OPTIMISATION

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def active_portfolio_optimization(normalized_data, original_data,portfolio_type_weights = 'EW'):

    # Get clusters from rule based algorithm
    binary_data = process_data_binary_tranpose(normalized_data, original_data)
    clusters = rule_based_clusters(binary_data)

    #create portfolios from clusters based on type, here default is equally weighted portfolio - EW
    normalized_data_with_ticker = normalized_data.copy()
    normalized_data_with_ticker['Ticker'] = original_data['Ticker']
    portfolios = create_portfolios(normalized_data_with_ticker, clusters, portfolio_type_weights)
    for key, value in portfolios.items():
      print(f"{key}: {value}")
    # one dummy output - EW_Cluster_0: {'K71U.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333}

    #optimise the portfolios created
    optimized_portfolios = {}
    metrics = {}
    for portfolio_cluster, ticker_weights in portfolios.items():
        tickers_in_each_cluster = list(ticker_weights.keys())
        print(f"Processing portfolio: {portfolio_cluster} with tickers: {tickers_in_each_cluster}")
        #cluster_prices = original_data['Ticker'].isin(tickers_in_each_cluster)  # Prices of the stocks in this portfolio cluster
        cluster_prices = original_data.loc[original_data['Ticker'].isin(tickers_in_each_cluster), 'Adj Close']
        cluster_prices = cluster_prices.values.reshape(-1,1)
        cluster_prices = pd.DataFrame(cluster_prices)
        print(f"The first rows in cluster_prices : {cluster_prices.head()}")
        # Predict future prices using LSTM
        forecasted_prices = lstm_forecast(cluster_prices, portfolio_cluster)
        forecasted_prices = np.resize(forecasted_prices, cluster_prices.shape)
        print(f"The lstm_forecast prices : {forecasted_prices}")
        # Calculate returns and covariance matrix for PSO optimization
        returns = cluster_prices.pct_change().mean() * 252  # Annualized returns
        cov_matrix = cluster_prices.pct_change().cov()

        # Optimize the portfolio using PSO
        optimal_weights = pso_optimization(returns, cov_matrix)

        # Calculate accuracy metrics for LSTM forecast
        mae = mean_absolute_error(cluster_prices, forecasted_prices)
        mse = mean_squared_error(cluster_prices, forecasted_prices)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((cluster_prices - forecasted_prices) / cluster_prices)) * 100  # MAPE in percentage

        # Store the metrics for this portfolio
        metrics[portfolio_cluster] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'MAPE': mape
        }

        # Store the optimized portfolio and forecasted prices
        optimized_portfolios[portfolio_cluster] = {
            'weights': optimal_weights,
            'forecasted_prices': forecasted_prices,
            'metrics': metrics[portfolio_cluster]  # Attach metrics
        }

        # Print the optimized weights and forecasted prices
        print(f"Optimized Weights for {portfolio_cluster}: {optimal_weights}")
        print(f"Forecasted Prices for {portfolio_cluster}: {forecasted_prices}")
        print(f"Metrics for {portfolio_cluster}: {metrics[portfolio_cluster]}")

    return optimized_portfolios

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### EW

In [34]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'EW')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


frozenset({'C52.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'C52.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'C52.SI'})
0: ['C52.SI', 'K71U.SI', 'OXMU.SI']
1: ['C2PU.SI', 'C52.SI', 'OXMU.SI']
2: ['C2PU.SI', 'K71U.SI', 'OXMU.SI']
3: ['C2PU.SI', 'K71U.SI', 'C52.SI']
EW_Cluster_0: {'C52.SI': 0.3333333333333333, 'K71U.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333}
EW_Cluster_1: {'C2PU.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333}
EW_Cluster_2: {'C2PU.SI': 0.3333333333333333, 'K71U.SI': 0.3333333333333333, 'OXMU.SI': 0.3333333333333333}
EW_Cluster_3: {'C2PU.SI': 0.3333333333333333, 'K71U.SI': 0.3333333333333333, 'C52.SI': 0.3333333333333333}
Processing portfolio: EW_Cluster_0 with tickers: ['C52.SI', 'K71U.SI', 'OXMU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 29s 106ms/step - loss

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - loss: 0.0440
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 0.0103
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 0.0089
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
The lstm_forecast prices : [[-0.54958296]
 [-0.54958946]
 [-0.5495947 ]
 ...
 [-0.5497091 ]
 [-0.54970247]
 [-0.5496972 ]]
portfolio_volatility : 0.0005912597671273402
portfolio_return : 0.0011584445530106293
portfolio_volatility : 0.001004557963033236
portfolio_return : 0.0019682122227144072
portfolio_volatility : 4.967786512719138e-05
portfolio_return : 9.733294139291087e-05
portfolio_volatility : 0.00023433505019951199
portfolio_return : 0.00045912841964880637
portfolio_volatility : 0.0006051030798653106
portfolio_return : 0.0011855675049321501
portfolio_volatility : 0.0007206009874938403
portfolio_return : 0.0014118604634847972
portfolio_volatility : 0.0007837018363468231
portfolio_return : 0.0015354928137785414
portfolio_volatility : 0.0009354606702256

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - loss: 0.0683
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0115
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 0.0102
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
The lstm_forecast prices : [[-0.5485388 ]
 [-0.54854614]
 [-0.548552  ]
 ...
 [-0.5486712 ]
 [-0.54866195]
 [-0.5486545 ]]
portfolio_volatility : 0.0010810669457047225
portfolio_return : 0.0009477817850167866
portfolio_volatility : 9.905205741351743e-05
portfolio_return : 8.683989105204836e-05
portfolio_volatility : 0.0011254925315019346
portfolio_return : 0.0009867301232067496
portfolio_volatility : 0.00023155567361188853
portfolio_return : 0.00020300708530457978
portfolio_volatility : 0.0007650611890145017
portfolio_return : 0.0006707364999477867
portfolio_volatility : 0.0006775742450288829
portfolio_return : 0.0005940358550286132
portfolio_volatility : 0.001084368373351361
portfolio_return : 0.0009506761783755576
portfolio_volatility : 3.069683929777246e-

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - loss: 0.0092
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.0047
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.0039
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
The lstm_forecast prices : [[-0.5182071 ]
 [-0.5180985 ]
 [-0.51802266]
 ...
 [-0.51908123]
 [-0.5192075 ]
 [-0.51931244]]
portfolio_volatility : 3.538481131709796e-05
portfolio_return : -9.48234428408772e-05
portfolio_volatility : 0.0007393537976699336
portfolio_return : -0.0019813041235199175
portfolio_volatility : 0.0002387266080718935
portfolio_return : -0.0006397343389016048
portfolio_volatility : 0.0001536968274355079
portfolio_return : -0.0004118733939415654
portfolio_volatility : 0.00016438344658985452
portfolio_return : -0.0004405111620354358
portfolio_volatility : 3.4366563393704854e-05
portfolio_return : -9.209476434386758e-05
portfolio_volatility : 0.0005954955195101961
portfolio_return : -0.001595795858574741
portfolio_volatility : 5.86236928171

- Metrics for EW_Cluster_3: {'MAE': 0.004657112110393389, 'MSE': 3.2552675393068766e-05, 'RMSE': 0.005705495192625156, 'MAPE': 0.8553877884080616}

#### MCW

In [35]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'MCW')

frozenset({'C52.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'C52.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'C52.SI'})
0: ['C52.SI', 'K71U.SI', 'OXMU.SI']
1: ['C2PU.SI', 'C52.SI', 'OXMU.SI']
2: ['C2PU.SI', 'K71U.SI', 'OXMU.SI']
3: ['C2PU.SI', 'K71U.SI', 'C52.SI']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


MCW_Cluster_0: {'C52.SI': 0.1312104468184239, 'K71U.SI': 0.42489144199997986, 'OXMU.SI': 0.4438981111815962}
MCW_Cluster_1: {'C2PU.SI': 0.27766177410645165, 'C52.SI': 0.42102632770806847, 'OXMU.SI': 0.3013118981854799}
MCW_Cluster_2: {'C2PU.SI': 0.20088608726589097, 'K71U.SI': 0.4301564639099436, 'OXMU.SI': 0.3689574488241654}
MCW_Cluster_3: {'C2PU.SI': 0.3382579035424287, 'K71U.SI': 0.2705982505134541, 'C52.SI': 0.39114384594411733}
Processing portfolio: MCW_Cluster_0 with tickers: ['C52.SI', 'K71U.SI', 'OXMU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - loss: 0.0822
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.0106
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 0.0079
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
The lstm_forecast prices : [[-0.5499665 ]
 [-0.5499726 ]
 [-0.54997796]
 ...
 [-0.5501091 ]
 [-0.55010456]
 [-0.5501007 ]]
portfol

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - loss: 0.1681
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 0.0120
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0104
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
The lstm_forecast prices : [[-0.5492827 ]
 [-0.5492886 ]
 [-0.5492938 ]
 ...
 [-0.5494105 ]
 [-0.54940546]
 [-0.5494007 ]]
portfolio_volatility : 2.253762261883657e-05
portfolio_return : 4.4157555802333734e-05
portfolio_volatility : 0.00024667851742340237
portfolio_return : 0.0004833127514193485
portfolio_volatility : 0.0006441650011907781
portfolio_return : 0.0012621008199071764
portfolio_volatility : 0.0002339702379548926
portfolio_return : 0.0004584136496252951
portfolio_volatility : 6.80669866703091e-05
portfolio_return : 0.00013336241417401276
portfolio_volatility : 0.0010007552281096656
portfolio_return : 0.0019607615930526657
portfolio_volatility : 0.00019206251968819618
portfolio_return : 0.000376304616245551
portfolio_volatility : 0.0010556150258760

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - loss: 0.0753
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - loss: 0.0095
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 0.0085
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
The lstm_forecast prices : [[-0.54949963]
 [-0.5495061 ]
 [-0.5495113 ]
 ...
 [-0.549611  ]
 [-0.5496018 ]
 [-0.5495941 ]]
portfolio_volatility : 0.0009026290668430898
portfolio_return : 0.0007913435810609327
portfolio_volatility : 0.0004804618740220596
portfolio_return : 0.00042122554426663275
portfolio_volatility : 0.0003214618192461291
portfolio_return : 0.00028182866756806514
portfolio_volatility : 0.0003960294940012923
portfolio_return : 0.0003472028649429827
portfolio_volatility : 0.0003771324998690862
portfolio_return : 0.0003306356885056356
portfolio_volatility : 0.0005105319675519695
portfolio_return : 0.0004475882843676367
portfolio_volatility : 0.0006829826488354577
portfolio_return : 0.0005987774546439318
portfolio_volatility : 0.000613088785418

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 0.0338
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - loss: 0.0045
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 0.0034
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
The lstm_forecast prices : [[-0.5179355 ]
 [-0.5178359 ]
 [-0.5177733 ]
 ...
 [-0.51886326]
 [-0.51897913]
 [-0.5190716 ]]
portfolio_volatility : 0.0007453823370680444
portfolio_return : -0.0019974592714422272
portfolio_volatility : 0.00038490343546175934
portfolio_return : -0.0010314558013237057
portfolio_volatility : 0.0004852685886616179
portfolio_return : -0.0013004121420083334
portfolio_volatility : 0.0003860934125634647
portfolio_return : -0.001034644675913832
portfolio_volatility : 0.00068308588522211
portfolio_return : -0.0018305186033723633
portfolio_volatility : 1.009610379260776e-05
portfolio_return : -2.705531795308217e-05
portfolio_volatility : 0.0003646781494975575
portfolio_return : -0.0009772565227015757
portfolio_volatility : 0.000522542752

- Metrics for MCW_Cluster_3: {'MAE': 0.0048060208658446246, 'MSE': 3.4582090096632244e-05, 'RMSE': 0.00588065388342421, 'MAPE': 0.882730260343261}

#### GMV

In [36]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'GMV')

frozenset({'C52.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'C52.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'C52.SI'})
0: ['C52.SI', 'K71U.SI', 'OXMU.SI']
1: ['C2PU.SI', 'C52.SI', 'OXMU.SI']
2: ['C2PU.SI', 'K71U.SI', 'OXMU.SI']
3: ['C2PU.SI', 'K71U.SI', 'C52.SI']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


GMV_Cluster_0: {'C52.SI': 9.017911069511968e-16, 'K71U.SI': 6.143106534793197e-16, 'OXMU.SI': -3.5132977726386065e-15}
GMV_Cluster_1: {'C2PU.SI': 6.813276498767571e-15, 'C52.SI': -5.679386269279177e-15, 'OXMU.SI': -7.713770162093047e-15}
GMV_Cluster_2: {'C2PU.SI': 3.0158880825412335e-15, 'K71U.SI': 8.319378731199472e-16, 'OXMU.SI': 9.711918938984257e-16}
GMV_Cluster_3: {'C2PU.SI': -3.366436271514299e-17, 'K71U.SI': -4.512797562569167e-15, 'C52.SI': -4.321775796660673e-15}
Processing portfolio: GMV_Cluster_0 with tickers: ['C52.SI', 'K71U.SI', 'OXMU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - loss: 0.0558
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - loss: 0.0072
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0065
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
The lstm_forecast prices : [[-0.5500592 ]
 [-0.55006564]
 [-0.5500714 ]
 ...
 [-0.5502156 ]
 [-0.55021065]
 [-0.55020624]]
portfolio_volatility : 0.00012108538599588089
portfolio_return : 0.000875335541761862
portfolio_volatility : 1.2566462550046711e-06
portfolio_return : 9.084392153359847e-06
portfolio_volatility : 0.00017452213631516468
portfolio_return : 0.0012616339080429493
portfolio_volatility : 0.00013998890330141047
portfolio_return : 0.001011990518130384
portfolio_volatility : 0.00023239824791057134
portfolio_return : 0.0016800247574569214
portfolio_volatility : 9.057689400984086e-05
portfolio_return : 0.0006547873134079776
portfolio_volatility : 3.633433588471515e-05
portfolio_return : 0.00026266370069866684
portfolio_volatility : 0.0002132239202

...

173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - loss: 0.0638
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 0.0112
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 0.0093
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
The lstm_forecast prices : [[-0.5492285 ]
 [-0.54923534]
 [-0.54924065]
 ...
 [-0.5493407 ]
 [-0.54933196]
 [-0.54932517]]
portfolio_volatility : 0.000507562883258109
portfolio_return : 0.0009944587643388421
portfolio_volatility : 0.00022576807111843102
portfolio_return : 0.0004423432926978349
portfolio_volatility : 0.00025773426219508603
portfolio_return : 0.00050497407191213
portfolio_volatility : 0.0008077623036147979
portfolio_return : 0.0015826340515206197
portfolio_volatility : 0.0004676768849052246
portfolio_return : 0.0009163108501694368
portfolio_volatility : 0.0005714677021179225
portfolio_return : 0.0011196663185057026
portfolio_volatility : 0.00015032717594482033
portfolio_return : 0.0002945333131473525
portfolio_volatility : 0.0010436167344559082

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 0.1205
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - loss: 0.0130
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.0108
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
The lstm_forecast prices : [[-0.5489119 ]
 [-0.5489167 ]
 [-0.5489213 ]
 ...
 [-0.54902416]
 [-0.5490194 ]
 [-0.5490147 ]]
portfolio_volatility : 0.0005369299003377797
portfolio_return : 0.00047073160583898066
portfolio_volatility : 0.0002925266818271623
portfolio_return : 0.0002564609544013499
portfolio_volatility : 0.0009981958090646294
portfolio_return : 0.0008751278627752583
portfolio_volatility : 0.0007433296088027186
portfolio_return : 0.0006516842146418728
portfolio_volatility : 0.0004895349983488481
portfolio_return : 0.0004291800395957976
portfolio_volatility : 0.0006756526367148641
portfolio_return : 0.0005923511625447665
portfolio_volatility : 0.0010358193806385245
portfolio_return : 0.0009081128096989364
portfolio_volatility : 0.000941704303184475

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 0.0332
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 0.0056
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.0041
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
The lstm_forecast prices : [[-0.51700425]
 [-0.5168918 ]
 [-0.5168058 ]
 ...
 [-0.5177796 ]
 [-0.5179106 ]
 [-0.5180244 ]]
portfolio_volatility : 0.0004099319315275182
portfolio_return : -0.0010985266172400768
portfolio_volatility : 0.0005564659234715924
portfolio_return : -0.0014912052014166859
portfolio_volatility : 0.00015080911752782903
portfolio_return : -0.0004041349721391151
portfolio_volatility : 0.0004368218848730285
portfolio_return : -0.00117058572562989
portfolio_volatility : 0.000530145320939744
portfolio_return : -0.001420671826874988
portfolio_volatility : 0.0003742525653009872
portfolio_return : -0.0010029138326003181
portfolio_volatility : 0.0005978489833160762
portfolio_return : -0.0016021026193675314
portfolio_volatility : 0.000606013455117

- Metrics for GMV_Cluster_3: {'MAE': 0.0047469111093427245, 'MSE': 3.3709590026082156e-05, 'RMSE': 0.005805996040825567, 'MAPE': 0.8718782356212688}

####MSR

In [37]:
optimized_portfolios_set1 = active_portfolio_optimization(normalized_data, original_data, portfolio_type_weights = 'MSR')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


frozenset({'C52.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'C52.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'OXMU.SI'})
frozenset({'C2PU.SI', 'K71U.SI', 'C52.SI'})
0: ['C52.SI', 'K71U.SI', 'OXMU.SI']
1: ['C2PU.SI', 'C52.SI', 'OXMU.SI']
2: ['C2PU.SI', 'K71U.SI', 'OXMU.SI']
3: ['C2PU.SI', 'K71U.SI', 'C52.SI']
MSR_Cluster_0: {'C52.SI': 0.00438154914916516, 'K71U.SI': 0.05776671445403798, 'OXMU.SI': 0.0030716394468776253}
MSR_Cluster_1: {'C2PU.SI': -0.0009750903144902756, 'C52.SI': 0.017754646807605277, 'OXMU.SI': 0.013069133098175834}
MSR_Cluster_2: {'C2PU.SI': -0.004010857786348803, 'K71U.SI': -0.021696935608734605, 'OXMU.SI': -0.013965208259085463}
MSR_Cluster_3: {'C2PU.SI': -0.0006948749674130618, 'K71U.SI': 0.0027647140595214886, 'C52.SI': 0.0024170604552142534}
Processing portfolio: MSR_Cluster_0 with tickers: ['C52.SI', 'K71U.SI', 'OXMU.SI']
The first rows in cluster_prices :           0
0 -0.536907
1 -0.536980
2 -0.537272
3 -0.537126
4 -0.537418


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 0.0597
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 0.0080
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.0073
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
The lstm_forecast prices : [[-0.5499476 ]
 [-0.54995364]
 [-0.5499586 ]
 ...
 [-0.550069  ]
 [-0.55006236]
 [-0.550057  ]]
portfolio_volatility : 1.2471204285347286e-05
portfolio_return : 9.015529223244729e-05
portfolio_volatility : 0.00013200729729353502
portfolio_return : 0.0009542908761663979
portfolio_volatility : 7.530583929073116e-05
portfolio_return : 0.0005443916876610221
portfolio_volatility : 0.0002592357748826688
portfolio_return : 0.0018740352981878126
portfolio_volatility : 9.88487945758367e-05
portfolio_return : 0.0007145855169961673
portfolio_volatility : 0.00018388500389306445
portfolio_return : 0.001329318795829691
portfolio_volatility : 1.4563854357034952e-05
portfolio_return : 0.00010528322009222251
portfolio_volatility : 0.00011

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - loss: 0.1106
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - loss: 0.0109
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - loss: 0.0094
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
The lstm_forecast prices : [[-0.5499379]
 [-0.5499447]
 [-0.5499499]
 ...
 [-0.5500512]
 [-0.550042 ]
 [-0.550035 ]]
portfolio_volatility : 0.0007029786589156108
portfolio_return : 0.0013773333542718735
portfolio_volatility : 6.168208206022619e-05
portfolio_return : 0.00012085258621298053
portfolio_volatility : 0.000138026714118076
portfolio_return : 0.00027043324107253763
portfolio_volatility : 0.0007011162242889397
portfolio_return : 0.0013736843198397018
portfolio_volatility : 0.0003689207649739462
portfolio_return : 0.0007228197730297087
portfolio_volatility : 0.0009724140935814103
portfolio_return : 0.0019052333214776987
portfolio_volatility : 0.00039057437554049115
portfolio_return : 0.0007652453000289523
portfolio_volatility : 0.00016043861656281385


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - loss: 0.1770
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.0137
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.0112
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
The lstm_forecast prices : [[-0.54904866]
 [-0.5490533 ]
 [-0.5490571 ]
 ...
 [-0.54913515]
 [-0.5491297 ]
 [-0.5491252 ]]
portfolio_volatility : 1.035042209282771e-05
portfolio_return : 9.074314560993775e-06
portfolio_volatility : 0.00022966787366779145
portfolio_return : 0.00020135203294368778
portfolio_volatility : 3.651243332166696e-05
portfolio_return : 3.2010801335117294e-05
portfolio_volatility : 0.0009712920926432122
portfolio_return : 0.0008515411159278148
portfolio_volatility : 0.0011147372074023576
portfolio_return : 0.0009773008271635835
portfolio_volatility : 0.001090328210061757
portfolio_return : 0.0009559012245193056
portfolio_volatility : 0.0011416407949919126
portfolio_return : 0.001000887461062895
portfolio_volatility : 0.000547637711203208

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


173/173 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - loss: 0.0199
Epoch 2/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 0.0041
Epoch 3/3
173/173 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 0.0032
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
The lstm_forecast prices : [[-0.5165709 ]
 [-0.51645756]
 [-0.5163803 ]
 ...
 [-0.5173555 ]
 [-0.51748717]
 [-0.5175949 ]]
portfolio_volatility : 0.0006901154190110933
portfolio_return : -0.001849356194152891
portfolio_volatility : 0.0002689546196298317
portfolio_return : -0.0007207387026234066
portfolio_volatility : 2.5770047357923137e-06
portfolio_return : -6.9058008837536805e-06
portfolio_volatility : 0.000123156122352583
portfolio_return : -0.00033003108095595174
portfolio_volatility : 0.0003811753208319048
portfolio_return : -0.0010214652813421159
portfolio_volatility : 0.0003172314468500468
portfolio_return : -0.0008501098874922937
portfolio_volatility : 0.00010065232144153747
portfolio_return : -0.0002697258878529463
portfolio_volatility : 5.991046034

- Metrics for MSR_Cluster_3: {'MAE': 0.004838376151140671, 'MSE': 3.4829945859603045e-05, 'RMSE': 0.005901690085018277, 'MAPE': 0.8886607884233877}

### Results and analysis for experiment - rule based clustering with PSO optimization (result from one of the runs):
- Cluster 3 best here

- 0: ['OXMU.SI', 'K71U.SI', 'C2PU.SI']
- 1: ['K71U.SI', 'C52.SI', 'C2PU.SI']
- 2: ['OXMU.SI', 'C52.SI', 'C2PU.SI']
- 3: ['K71U.SI', 'OXMU.SI', 'C52.SI']

#### Max return
- Processing portfolio: GMV_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
- max return - /content/model_weights_GMV_Cluster_3.weights.h5
- max return - GMV weights - GMV_Cluster_3: {'K71U.SI': 9.017911069511968e-16, 'OXMU.SI': 6.143106534793197e-16, 'C52.SI': -3.5132977726386065e-15}

#### Min risk
- Processing portfolio: MCW_Cluster_3 with tickers: ['K71U.SI', 'OXMU.SI', 'C52.SI']
- min risk - /content/model_weights_MCW_Cluster_3.weights.h5
- min risk - MCW weights - MCW_Cluster_3: {'K71U.SI': 0.3459780986862702, 'OXMU.SI': 0.36653566994865966, 'C52.SI': 0.2874862313650701}
